<a href="https://www.kaggle.com/code/suryan5h/advertising-dataset-sales-prediction-using-pyspark?scriptVersionId=110417273" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%pip install pyspark
%pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 13.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=6f1f7f2f7a84a0f3e0316862f85f1c21f3d4e0ac241883eddccf712447e62857
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark import SparkContext, SparkConf

In [3]:
sc = SparkContext.getOrCreate()
sc

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/08 15:42:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[*] appName=pyspark-shell>

In [4]:
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)
df = spark.read.csv('/kaggle/input/advertising-data/Advertising.csv',header=True,inferSchema=False)
df.head(5)

22/11/08 15:42:09 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TV, Radio, Newspaper, Sales
 Schema: _c0, TV, Radio, Newspaper, Sales
Expected: _c0 but found: 
CSV file: file:///kaggle/input/advertising-data/Advertising.csv


[Row(_c0='1', TV='230.1', Radio='37.8', Newspaper='69.2', Sales='22.1'),
 Row(_c0='2', TV='44.5', Radio='39.3', Newspaper='45.1', Sales='10.4'),
 Row(_c0='3', TV='17.2', Radio='45.9', Newspaper='69.3', Sales='9.3'),
 Row(_c0='4', TV='151.5', Radio='41.3', Newspaper='58.5', Sales='18.5'),
 Row(_c0='5', TV='180.8', Radio='10.8', Newspaper='58.4', Sales='12.9')]

In [5]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- TV: string (nullable = true)
 |-- Radio: string (nullable = true)
 |-- Newspaper: string (nullable = true)
 |-- Sales: string (nullable = true)



In [6]:
print(df.count(),len(df.columns))

200 5


In [7]:
df.head(2)

22/11/08 15:42:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , TV, Radio, Newspaper, Sales
 Schema: _c0, TV, Radio, Newspaper, Sales
Expected: _c0 but found: 
CSV file: file:///kaggle/input/advertising-data/Advertising.csv


[Row(_c0='1', TV='230.1', Radio='37.8', Newspaper='69.2', Sales='22.1'),
 Row(_c0='2', TV='44.5', Radio='39.3', Newspaper='45.1', Sales='10.4')]

In [8]:
df = df.drop('_c0')
df.printSchema()

root
 |-- TV: string (nullable = true)
 |-- Radio: string (nullable = true)
 |-- Newspaper: string (nullable = true)
 |-- Sales: string (nullable = true)



In [9]:
from pyspark.sql.functions import col,countDistinct
from pyspark.sql.functions import when, count,isnull
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType

# Check Null Values

In [10]:
df.select([count(when(isnull(c),c)).alias(c) for c in df.columns]).show()

+---+-----+---------+-----+
| TV|Radio|Newspaper|Sales|
+---+-----+---------+-----+
|  0|    0|        0|    0|
+---+-----+---------+-----+



# Change required columns from String type to numerical

In [11]:
df = df.withColumn('TV',df['TV'].cast(FloatType()))
df = df.withColumn('Radio',df['Radio'].cast(FloatType()))
df = df.withColumn('Newspaper',df['Newspaper'].cast(FloatType()))
df = df.withColumn('Sales',df['Sales'].cast(FloatType()))

In [12]:
df.printSchema()

root
 |-- TV: float (nullable = true)
 |-- Radio: float (nullable = true)
 |-- Newspaper: float (nullable = true)
 |-- Sales: float (nullable = true)



In [13]:
df.head(5)

[Row(TV=230.10000610351562, Radio=37.79999923706055, Newspaper=69.19999694824219, Sales=22.100000381469727),
 Row(TV=44.5, Radio=39.29999923706055, Newspaper=45.099998474121094, Sales=10.399999618530273),
 Row(TV=17.200000762939453, Radio=45.900001525878906, Newspaper=69.30000305175781, Sales=9.300000190734863),
 Row(TV=151.5, Radio=41.29999923706055, Newspaper=58.5, Sales=18.5),
 Row(TV=180.8000030517578, Radio=10.800000190734863, Newspaper=58.400001525878906, Sales=12.899999618530273)]

# Vector Assembler

In [14]:
trainingcols = ['TV','Radio','Newspaper']

In [15]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = trainingcols,outputCol = 'features')
assembler

VectorAssembler_c9adb78188ea

# Initialize the Random Forest Regressor

In [16]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(featuresCol='features',labelCol = 'Sales',seed=42)
rf

RandomForestRegressor_f66f0b35895e

# Splitting the dataset into training and testing set

In [17]:
train, test = df.randomSplit([0.7,0.3],seed=2020)

In [18]:
print(train.count(),len(train.columns))
print(test.count(),len(test.columns))

154 4
46 4


# Creating the pipeline and input the stages

In [19]:
from pyspark.ml import Pipeline
stages = [assembler,rf]
pipeline = Pipeline(stages=stages)
#Training
pipelineModel = pipeline.fit(train)

# Model Predictions

In [20]:
predictions = pipelineModel.transform(test)
rfModel = pipelineModel.stages[1]
print(rfModel)

RandomForestRegressionModel: uid=RandomForestRegressor_f66f0b35895e, numTrees=20, numFeatures=3


In [21]:
predictions.show()

+-----+-----+---------+-----+--------------------+------------------+
|   TV|Radio|Newspaper|Sales|            features|        prediction|
+-----+-----+---------+-----+--------------------+------------------+
| 11.7| 36.9|     45.2|  7.3|[11.6999998092651...| 8.866845605506663|
| 17.2|  4.1|     31.6|  5.9|[17.2000007629394...|  7.89513063393611|
| 17.2| 45.9|     69.3|  9.3|[17.2000007629394...|14.260972335161986|
| 18.7| 12.1|     23.4|  6.7|[18.7000007629394...| 8.311445538969938|
| 18.8| 21.7|     50.4|  7.0|[18.7999992370605...|10.088001451179235|
| 23.8| 35.1|     65.9|  9.2|[23.7999992370605...|12.197597236456696|
| 25.0| 11.0|     29.7|  7.2|[25.0,11.0,29.700...|   8.4318066701347|
| 25.6| 39.0|      9.3|  9.5|[25.6000003814697...| 9.269007164320666|
| 39.5| 41.1|      5.8| 10.8|[39.5,41.09999847...|12.611447722254656|
| 48.3| 47.0|      8.5| 11.6|[48.2999992370605...|11.987850456507797|
| 53.5|  2.0|     21.4|  8.1|[53.5,2.0,21.3999...| 9.093469669641888|
| 76.4| 26.7|     22

# Feature Importance

In [22]:
rfModel.featureImportances

SparseVector(3, {0: 0.5292, 1: 0.2944, 2: 0.1763})

# Use Regression Metrics to evaluate this Random Forest model

In [23]:
from pyspark.mllib.evaluation import RegressionMetrics
#squared error
predandobs = predictions.select(['prediction','Sales']).withColumn('label', F.col('Sales').cast(FloatType())).orderBy('prediction')

In [24]:
metrics = RegressionMetrics(predandobs.rdd.map(tuple))
print("RMSE: "+str(metrics.rootMeanSquaredError))
print("R2: "+str(metrics.r2))

/opt/conda/lib/python3.7/site-packages/pyspark/sql/context.py:159: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


RMSE: 2.4974805334087393
R2: 0.8155872202218665
